In [1]:
import sys
sys.path.append('/public/src')
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

from factor_evaluation_server import FactorEvaluation, DataService
ds = DataService()
df = ds['ETHUSDT_15m_2020_2025']['2021-10-01':]

In [2]:
path="/public/data/factor_data/ETH_15m_factor_data.txt"
factors=pd.read_csv(path, sep='|')
factors.head()

,Unnamed: 0,ret_bollinger_rsi_signals,c_chu016,c_chu009,c_chu011,ret_hv_ratio_signals,ret_vao_signals,c_chu028,ret_ma120_cci_cross_sig_price,ret_ma20_volume_cross_signals,...,c_chu059,ret_mfi_sig_price,c_hide_009,c_chu044,c_hide_014,ret_rsi_ma120_cross_sig_price,c_hide_006,c_chu037,c_chu004,c_hide_010
0,2019-12-31 16:00:00,0,NaN,NaN,-0.0235,0,0,NaN,0,0,...,NaN,0,0.0000,7111059.2053,0.0000,0,0.0000,NaN,NaN,0.0000
1,2019-12-31 16:15:00,0,NaN,-0.0000,0.0650,0,0,NaN,0,0,...,NaN,0,0.0000,2926088.3661,0.0000,0,0.0000,0.0000,NaN,0.0000
2,2019-12-31 16:30:00,0,NaN,-0.0001,0.3217,0,0,NaN,0,0,...,NaN,0,0.0000,2166516.5598,0.0000,0,0.0000,0.0001,NaN,0.0000
3,2019-12-31 16:45:00,0,NaN,0.0000,0.1571,0,0,NaN,0,0,...,NaN,0,0.0000,135422.1719,0.0027,0,0.0000,0.0000,NaN,0.0000
4,2019-12-31 17:00:00,0,NaN,-0.0001,-0.0850,0,0,NaN,0,0,...,NaN,0,0.0000,12405.6168,0.0008,0,0.0000,0.0001,NaN,0.0000


In [3]:
for i in list(factors.columns):
    print(i)

Unnamed: 0
ret_bollinger_rsi_signals
c_chu016
c_chu009
c_chu011
ret_hv_ratio_signals
ret_vao_signals
c_chu028
ret_ma120_cci_cross_sig_price
ret_ma20_volume_cross_signals
c_chu026
c_chu015
c_chu029
c_chu012
c_chu058
c_hide_028
c_chu045
ret_ma_bbi_rsi_sig_price
c_chu023
c_hide_001
c_hide_032
c_chu019
c_chu032
c_chu049
c_chu051
c_hide_008
ret_macd_sig_price
c_chu022
ret_ma_arrangement_sig
ret_ma120_bolling_cross_sig_price
c_hide_019
c_hide_026
c_chu033
c_hide_030
ret_skdj_sig_price
ret_ma120_bbi_signals
c_chu043
ret_williams_r_sig_price
c_chu042
ret_ma20_rsi_macd_cross_sig_price
c_chu053
c_chu005
c_chu052
ret_cci_fibonacci_signals
ret_ma_cci_sig
c_chu021
c_hide_005
ret_ao_signals
ret_ma_atr_cross_sig_price
c_chu006
c_hide_021
c_hide_029
ret_rma_cross_sig_price
c_hide_002
ret_rsi_bb_ma_signal
c_chu010
c_chu048
c_chu041
c_hide_016
ret_kc_strategy
c_chu047
c_chu056
ret_ma120_macd_1_cross_sig_price
c_chu025
c_hide_011
c_chu018
c_hide_018
c_hide_031
c_chu054
ret_td_signals
c_chu001
c_chu040
c_

In [4]:
factors.index

RangeIndex(start=0, stop=192421, step=1)

In [5]:
def factor(df, ema_period=20, atr_period=10, multiplier=2):
    '''凯尔特纳通道：基于ATR的波动通道'''
    ema = df['close'].ewm(span=ema_period, adjust=False).mean()
    
    # 计算ATR
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift(1))
    low_close = abs(df['low'] - df['close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = true_range.ewm(span=atr_period, adjust=False).mean()
    
    # 计算通道宽度
    channel_width = multiplier * atr
    
    factor = (df['close'] - (ema + channel_width)) / (2 * channel_width)
    
    return factor

In [7]:
sig=factor(df)

In [8]:
# sig的index修改为arrange
sig = sig.reset_index(drop=True)

In [9]:
factors['sig']=sig

In [10]:
corr_matrix=factors.corr()

In [13]:
corr_matrix.iloc[-1,:]

# 我想看到这一行的全部数值
corr_matrix.iloc[-1,:].sort_values(ascending=False)

sig                                  1.0000
c_chu018                             0.0181
c_chu007                             0.0135
close_time                           0.0090
open_time                            0.0090
ret_williams_r_sig_price             0.0087
c_chu056                             0.0081
c_hide_031                           0.0045
ret_kc_strategy                      0.0038
c_chu061                             0.0033
c_hide_029                           0.0033
c_chu050                             0.0032
c_chu026                             0.0031
ret_skdj_sig_price                   0.0029
c_hide_005                           0.0028
c_chu044                             0.0027
ret_bollinger_rsi_signals            0.0027
c_hide_011                           0.0024
c_hide_030                           0.0023
c_hide_017                           0.0019
ret_macd_cross_signal                0.0019
c_chu033                             0.0018
c_chu004                        